### Facebook

In [ ]:
import requests
import pandas as pd
import numpy as np
import glob
import os
import re
import time
import pandas as pd
from apify_client import ApifyClient
from bs4 import BeautifulSoup
import re
import json
from requests.exceptions import HTTPError, ConnectionError
from urllib3.exceptions import ProtocolError
from langchain.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI


In [ ]:
# Initialize the ApifyClient with your API token
client = ApifyClient("apify_api_yNR85etaHpLtBzPoVozVVXUsCZe54u2Ffog1")

# Function to validate Singapore phone numbers (MUST have country code)
def validate_singapore_number(phone):
    if not phone:
        return None
    
    # Remove all spaces, dashes, parentheses
    cleaned = re.sub(r'[\s\-\(\)]', '', str(phone))
    
    # MUST have country code: +65XXXXXXXX or 65XXXXXXXX
    # First digit after country code must be 6, 8, or 9
    # Total of 8 digits after country code
    if re.match(r'^\+?65[689]\d{7}$', cleaned):
        return phone  # Return original format
    
    # Not a valid Singapore number with country code
    return None

# Prepare the Actor input
run_input = {
    "pages": [
        "https://www.facebook.com/KPECTHub/",
    ],
    "language": "en-US",
}

# Run the Actor and wait for it to finish
run = client.actor("oJ48ceKNY7ueGPGL0").call(run_input=run_input)

# Collect results
results = []
for item in client.dataset(run["defaultDatasetId"]).iterate_items():
    # Extract phone from multiple possible fields
    raw_phone = item.get('phone', None) or item.get('wa_number', None)
    
    # Validate it's a Singapore number WITH country code
    phone = validate_singapore_number(raw_phone)
    
    # Extract email
    email = item.get('email', None)
    
    # Extract website from the websites list (take first non-Google Maps link if available)
    websites = item.get('websites', [])
    website = None
    if websites:
        # Filter out Google Maps links and take the first real website
        real_websites = [w for w in websites if 'maps.google.com' not in w]
        website = real_websites[0] if real_websites else websites[0]
    
    results.append({
        'facebook_url': item.get('facebookUrl', None),
        'page_name': item.get('pageName', None),
        'phone': phone,  # Only Singapore numbers WITH country code or None
        'email': email,
        'website': website,
        'address': item.get('address', None)
    })

# Create DataFrame
df = pd.DataFrame(results)

df